## Análisis de géneros y títulos

### Retos

1. Extraer el año de la columna 'title' en una nueva columna 'year' (Ej: "Toy Story (1995)" → 1995). Usar una expresión regular con regexp_extract.
2. Asegurar que los datos no tienen películas repetidas por movieId.
3. Calcular el top 3 de películas más recientes por género, usar:
    - Window.partitionBy().orderBy()
    - row_number() de pyspark.sql.functions
4. Guardar el resultado final con estas columnas: [movieId; title; year; genre]. Particionar por género. Leer solo las películas del género 'Horror'

### Resultados

In [10]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.window import Window

spark = SparkSession.builder.appName("PySpark01-challenge").getOrCreate()
df = spark.read.option("header", True).option("inferSchema", True).csv("../../data/movies.csv")

Apartado 1

In [ ]:
df_year = df.withColumn('year', F.regexp_extract(F.col('title'), r"\((\d{4})\)", 1))

Apartado 2

In [ ]:
df.groupBy("movieId").agg(F.count("*").alias("repetitions")).filter("repetitions > 1").show()

Apartado 3

In [ ]:
df_exploded = df_year.withColumn("genre", F.explode(F.split(F.col("genres"), "\|")))
window_spec = Window.partitionBy("genre").orderBy(F.col("year").desc())
df_top3 = df_exploded.withColumn("rank", F.row_number().over(window_spec)) \
                     .filter(F.col("rank") <= 3) \
                     .select("movieId", "title", "year", "genre")
df_top3.show()

Apartado 4

In [ ]:
df_top3.write.partitionBy("genre").parquet("data/partition_by_genre.parquet")
horror_data = spark.read.parquet("./data/partition_by_genre.parquet/genre=Horror")
horror_data.show()